<a href="https://colab.research.google.com/github/klovak96/test_bot_tg/blob/main/notes_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SQL основы**

In [ ]:
from
on
join
where
group by
having
over
select
order by

### *Примеры из итоговой*

In [ ]:
/* 1.
 * Выведите название самолетов, которые имеют менее 50 посадочных мест?
 */
explain analyze
select a.aircraft_code, a.model
from (select s.aircraft_code
	  from seats s
	  group by s.aircraft_code
	  having count(s.seat_no) < 50) s
join aircrafts a on s.aircraft_code = a.aircraft_code


/* 2.
 * Выведите процентное изменение ежемесячной суммы бронирования билетов, округленной до сотых.
 */
explain analyze

with recursive r as (
	select min(date_trunc('month', book_date)) as x
	from bookings
	union
	select x + interval '1 month' as x
	from r
	where x <(select max(date_trunc('month', book_date)) from bookings))
select concat(date_part('year', x::date), '-', date_part('month', x::date)),
       round(((b.sum - lag(b.sum) over (order by x))/lag(b.sum) over (order by x))*100, 2) as "изменение в %"
from r
left join(
	select date_trunc('month', book_date) as dt, sum(total_amount)
	from bookings
	group by date_trunc('month', book_date)) b on b.dt = r.x
order by x::date



/* 3.
 * Выведите названия самолетов не имеющих бизнес - класс. Решение должно быть через функцию array_agg.
 */

select a.aircraft_code, a.model
from (select s.aircraft_code, array_agg(s.fare_conditions)
	  from seats s
	  group by s.aircraft_code) s
join aircrafts a on s.aircraft_code = a.aircraft_code
where not 'Business' = any(s.array_agg)

/* 4.
 * Вывести накопительный итог количества мест в самолетах по каждому аэропорту на каждый день,
 * учитывая только те самолеты, которые летали пустыми и только те дни, где из одного аэропорта таких самолетов вылетало более одного.
 * В результате должны быть код аэропорта, дата, количество пустых мест в самолете и накопительный итог.
 */

with cte as(
	select f.flight_id, f.actual_departure, f.departure_airport, f.aircraft_code
	from flights f
	left join boarding_passes bp on bp.flight_id = f.flight_id
	where bp.flight_id is null and f.actual_departure is not null),
cte1 as(
	select s.aircraft_code, count(s.seat_no) as cs
	from seats s
	group by s.aircraft_code),
cte2 as(
	select cte.actual_departure::date, cte.departure_airport, array_agg(cte.flight_id)
	from cte
	group by cte.departure_airport, cte.actual_departure::date
	having count(cte.flight_id) > 1)
select cte.departure_airport as "код аэропорта",
	   cte.actual_departure as "дата",
	   cte1.cs as "количество пустых мест в самолете",
	   sum(cte1.cs) over (partition by cte.departure_airport, cte.actual_departure::date order by cte.actual_departure) as "накопительный итог"
from cte2
join cte on cte.flight_id =any(cte2.array_agg)
join cte1 on cte.aircraft_code = cte1.aircraft_code


/* 5.
 * Найдите процентное соотношение перелетов по маршрутам от общего количества перелетов.
 * Выведите в результат названия аэропортов и процентное отношение.
 * Решение должно быть через оконную функцию.
 */
explain analyze
select a.airport_name as "Вылет",
	   b.airport_name as "Прилет",
	   (count(f.flight_id)/sum(count(f.flight_id)) over () )*100 as "% от всех полетов"
from flights f
left join airports a on a.airport_code = f.departure_airport
left join airports b on b.airport_code = f.arrival_airport
group by  a.airport_name, b.airport_name


/* 6.
 * Выведите количество пассажиров по каждому коду сотового оператора, если учесть, что код оператора - это три символа после +7
 */

select c.code, count(c.code)
from(select left(split_part(contact_data->>'phone', '+7',2),3) as code
	 from tickets) c
group by c.code


/* 7.
 * Классифицируйте финансовые обороты (сумма стоимости перелетов) по маршрутам:
 * До 50 млн - low
 * От 50 млн включительно до 150 млн - middle
 * От 150 млн включительно - high
 * Выведите в результат количество маршрутов в каждом полученном классе
 */

explain analyze
with cte1 as(
	select f.departure_airport, f.arrival_airport, sum(amount)
	from ticket_flights tf
	join flights f on f.flight_id = tf.flight_id
	group by f.departure_airport, f.arrival_airport),
cte2 as(
	select cte1.departure_airport, cte1.arrival_airport,
		   case
				when sum < 50000000 then 'low'
				when sum >= 50000000 and sum < 150000000 then 'middle'
				else 'high'
		   end	case1
		   from cte1)
select case1, count(case1)
from cte2
group by case1


/* 8.
 * Вычислите медиану стоимости перелетов, медиану размера бронирования и отношение
 * медианы бронирования к медиане стоимости перелетов, округленной до сотых
 */
explain analyze

select m1.ma as "медиану стоимости перелетов",
	   m2.mta as "медиану размера бронирования",
	   round((m2.mta/m1.ma)::numeric,2) as "отношение"
from (select percentile_cont(0.5) within group(order by tf.amount) as ma
	 from ticket_flights tf) m1,
	 (select percentile_cont(0.5) within group(order by b.total_amount) as mta
	 from bookings b) m2


/* 9.
 * Найдите значение минимальной стоимости полета 1 км для пассажиров.
 * То есть нужно найти расстояние между аэропортами и с учетом стоимости перелетов получить искомый результат
 * Для поиска расстояния между двумя точками на поверхности Земли используется модуль earthdistance.
 * Для работы модуля earthdistance необходимо предварительно установить модуль cube.
 * Установка модулей происходит через команду: create extension название_модуля.
 */
create extension cube

create extension earthdistance

explain analyze
with cte as(
	select f.flight_id, f.departure_airport, f.arrival_airport
	from flights f),
cte1 as(
	select cte.departure_airport, cte.arrival_airport, min(t.amount)
	from cte
	join ticket_flights t on t.flight_id = cte.flight_id
	group by cte.departure_airport, cte.arrival_airport)
select min/(earth_distance (ll_to_earth ( ad.latitude, ad.longitude), ll_to_earth ( aa.latitude, aa.longitude))/1000) as price1km
from cte1
join airports ad on ad.airport_code = cte1.departure_airport
join airports aa on aa.airport_code = cte1.arrival_airport
order by price1km
limit 1



### **Оконные функции**

In [ ]:
over () - все данные глобально в хаотичном порядке
over (partition by) - по группам отдельно в хаотичном порядке
over (order by) - все данные глобально в определенной последовательности передам данные в функцию
over (partition by  order by)  по группам отдельно в определенной последовательности передам данные в функцию


### **explain**

In [ ]:
https://tatiyants.com/pev/

explain (analyze, buffers) --2148.35 / 11
	select...

### **json**

In [ ]:
INSERT INTO orders (info)
VALUES
 (
'{"items": {"product": "Beer","qty": 6,"a":345}, "customer": "John Doe"}'
 ),
 (
'{ "customer": "Lily Bush", "items": {"product": "Diaper","qty": 24.5}}'
 ),
 (
'{ "customer": "Josh William", "items": {"product": "Toy Car","qty": 1}}'
 ),
 (
'{ "customer": "Mary Clark", "items": {"product": "Toy Train","qty": 2}}'
 );

select info->'items'->'qty', pg_typeof(info->'items'->'qty')
from orders

SyntaxError: invalid syntax (<ipython-input-1-1dd7c8e07fa6>, line 1)

### **представления / материализованные представления**

In [ ]:
обычное представление не хранит в себе данных

---------------------------------------------------

CREATE VIEW myview AS SELECT * FROM mytab;

SELECT * FROM mymatview
---------------------------------------------------

CREATE MATERIALIZED VIEW mymatview AS SELECT * FROM mytab;

REFRESH MATERIALIZED VIEW mymatview;

SELECT * FROM mymatview

### **cte**

In [ ]:
with название_cte (
	логика)
select
from ...
join название_cte

### **Создание таблиц**

In [ ]:

 https://dbdiagram.io/, https://sqldbm.com, https://pgmodeler.io

 create table author (
	author_id serial primary key,
	author_name varchar(100) not null,
	nick_name varchar (30),
	born_date date not null check(date_part('year', born_date) >= 1800 and date_part('year', age(born_date)) >= 18),
	city_id int2 not null references city(city_id),
	--language_id int2 not null references language(language_id),
	created_at timestamp not null default now(),
	created_user varchar(64) not null default current_user,
	deleted boolean not null default false)

uuid

create extension "uuid-ossp"

create table a (
	id uuid default uuid_generate_v4() primary key,
	val text)


-- если два первичных ключа

create table author_language (
	author_id int references author(author_id),
	language_id int2 references language(language_id),
	primary key (author_id, language_id))

-- первичный и вторичные ключи

create table contact( --список контактов контрагентов
	id uuid not null default uuid_generate_v1mc (),
	last_name varchar(20), --фамилия контакта
	first_name varchar(20), --имя контакта
	account_id uuid not null,--id контрагента
	PRIMARY KEY (id),
	FOREIGN KEY (account_id) REFERENCES account(id)
)

### **join**

In [ ]:
--left, right, inner, full, cross

INNER JOIN (JOIN) — каждая строка из первой (левой) таблицы, сопоставляется с каждой строкой из второй (правой) таблицы,
после чего, происходит проверка условия.

LEFT JOIN (LEFT OUTER JOIN) — важен порядок следования таблиц. Сначала происходит формирование таблицы соединением INNER JOIN.
Затем, в результат добавляются записи левой таблицы, не вошедшие в результат после INNER JOIN.
Для них, соответствующие записи из правой таблицы заполняются значениями NULL.

RIGHT JOIN (RIGHT OUTER JOIN) — важен порядок следования таблиц. Аналогично LEFT JOIN, но во главе вторая таблица.
Сначала происходит формирование таблицы соединением INNER JOIN.

Затем, в результат добавляются записи правой таблицы, не вошедшие в результат после INNER JOIN. Для них, соответствующие записи из левой таблицы заполняются значениями NULL.
FULL JOIN (FULL OUTER JOIN) — оператор FULL JOIN можно воспринимать как сочетание операторов INNER JOIN + LEFT JOIN + RIGHT JOIN.
Сначала происходит формирование таблицы соединением INNER JOIN.
Затем, в результат добавляются записи левой таблицы, не вошедшие прежде в результат.
Для них, соответствующие записи из правой таблицы заполняются значениями NULL.
Наконец, в таблицу добавляются значения не вошедшие в результат формирования из правой таблицы.
Для них, соответствующие записи из левой таблицы заполняются значениями NULL.

CROSS JOIN — каждая строка левой таблицы сопоставляется с каждой строкой правой таблицы.
В результате получается таблица со всеми возможными сочетаниями строк обеих таблиц (декартово произведение).


### **date_part / date_trunc**

In [ ]:
SELECT date_part('hour',TIMESTAMP '2017-03-18 10:20:30') h,
       date_part('minute',TIMESTAMP '2017-03-18 10:20:30') m,
       date_part('second',TIMESTAMP '2017-03-18 10:20:30') s;
10
20
30

SELECT DATE_TRUNC('hour', TIMESTAMP '2020-03-17 02:09:30');
--'2020-03-17 02:00:00'


## **lag/lead**

In [ ]:
SELECT  code,
LAG(code) OVER(ORDER BY code) prev_code,
LEAD(code) OVER(ORDER BY code) next_code

1 - 2
2 1 3
3 2 4
4 3 -

-lag следующее значение
-lead предыдущее

## **полезное**

In [ ]:
https://letsdocode.ru/sql-materials/info.php
https://letsdocode.ru/sql-main/info